### Stratified sampling

In [1]:
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipyleaflet
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
from humanize import naturalsize as size

pp = pprint.PrettyPrinter(depth=4)
ee.Initialize()


In [8]:
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterDate('2018-08-01', '2018-12-01')

global_water = (ee.ImageCollection('JRC/GSW1_0/YearlyHistory').filterDate('2015-01-01', '2018-01-01'))

# select a single grid for the analysis
grid = ee.FeatureCollection('users/samapriya/vector/la-subset').filterMetadata('Grids', 'equals', 'Grid_1')

print(f'Total number of LS-8 assets with filters: {global_water.size().getInfo()}')

Total number of LS-8 assets with filters: 1


In [9]:
pp.pprint(global_water.first().getInfo())
band_names_original = global_water.first().bandNames()
band_names_original.getInfo()

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [0.00026949458523585647,
                              0,
                              -180.00001488697754,
                              0,
                              -0.00026949458523585647,
                              84.00011474509029],
            'data_type': {'max': 255,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'id': 'waterClass'}],
 'id': 'JRC/GSW1_0/YearlyHistory/31',
 'properties': {'system:footprint': {'coordinates': [[...]],
                                     'geodesic': False,
                                     'type': 'Polygon'},
                'system:index': '31',
                'system:time_start': 1420070400000,
                'year': 2015},
 'type': 'Image',
 'version': 1655309684065679}


['waterClass']

In [10]:
def get_tile_layer_url(ee_image_object):
    """
    Returns a tile layer url from the earthengine server
    """
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
    return tile_url_template.format(**map_id)

In [13]:
map1 = ipyleaflet.Map(
    center=(29.2963,-90.3446),
    zoom=12,
    layout={'height': '500px'},
)

global_nw = global_water.median().updateMask(global_water.median().eq(0))
jrc_tile_url = get_tile_layer_url(global_water.median().visualize(min=0, max=3, bands=['waterClass'], palette='grey,brown,FFFFFF,blue'))
l8sr_tile_color = get_tile_layer_url(l8sr.median().visualize(min=100, max=3500, gamma=1.5, bands=['B5', 'B3', 'B2']))
left = ipyleaflet.TileLayer(url=jrc_tile_url, show_loading=True)
right = ipyleaflet.TileLayer(url=l8sr_tile_color, show_loading=True)
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
map1.add_control(control)
map1

Map(center=[29.2963, -90.3446], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [12]:
map1.layers

(TileLayer(attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', base=True, max_zoom=19, min_zoom=1, name='OpenStreetMap.Mapnik', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://tile.openstreetmap.org/{z}/{x}/{y}.png'),)